In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split

In [2]:
from functools import partial
from sklearn.model_selection import (
    cross_validate,
    KFold,
    ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

## The Validation Set Approach

In [3]:
Auto = load_data('Auto')
Auto_train , Auto_valid = train_test_split (Auto,
                                            test_size=196,
                                            random_state=0)

In [4]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train , X_train)
results = model.fit()

In [5]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean(( y_valid - valid_pred)**2)

23.61661706966988

In [6]:
def evalMSE(
        terms,
        response,
        train,
        test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train , X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean(( y_test - test_pred)**2)

In [7]:
MSE = np.zeros(3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([ poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [8]:
Auto_train , Auto_valid = train_test_split(Auto,
                                           test_size =196,
                                           random_state =3)
MSE = np.zeros (3)
for idx , degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE ([ poly('horsepower', degree)],
                        'mpg',
                        Auto_train,
                        Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

## Cross-Validation

In [9]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns =['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

In [15]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range (1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443029, 19.03320648])

### K-fold Cross Validation

In [18]:
cv_error = np.zeros (5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree
for i, d in enumerate(range (1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720065])

In [19]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis =1),
                         Auto['mpg'],
                         cv=validation)
results['test_score']

array([23.61661707])

In [20]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis =1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.4218450941091842)

---